In [2]:
!pip install transformers torch sentencepiece sacremoses

  Using cached pyyaml-6.0.3-cp313-cp313-win_amd64.whl.metadata (2.4 kB)
   ---------------------------------------- 0.0/12.0 MB ? eta -:--:--
    --------------------------------------- 0.3/12.0 MB ? eta -:--:--
   ----- ---------------------------------- 1.6/12.0 MB 6.4 MB/s eta 0:00:02
   -------- ------------------------------- 2.6/12.0 MB 8.0 MB/s eta 0:00:02
   ---------------- ----------------------- 5.0/12.0 MB 8.5 MB/s eta 0:00:01
   -------------------------- ------------- 7.9/12.0 MB 9.4 MB/s eta 0:00:01
   ---------------------------------- ----- 10.2/12.0 MB 9.8 MB/s eta 0:00:01
   ---------------------------------------- 12.0/12.0 MB 9.9 MB/s  0:00:01
   ---------------------------------------- 0.0/566.1 kB ? eta -:--:--
   ---------------------------------------- 566.1/566.1 kB 7.4 MB/s  0:00:00
   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ----------- ---------------------------- 0.8/2.7 MB 7.6 MB/s eta 0:00:01
   -------------------------------


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
# Add this cell BEFORE your preprocessing code

import pandas as pd
import re
from collections import Counter
from typing import Dict, List, Set

def analyze_trademark_vocabulary(csv_path: str, top_n: int = 100) -> Dict:
    """
    Analyze your trademark dataset to extract frequent words for lexicon building
    
    Args:
        csv_path: Path to trademark CSV file
        top_n: How many top words to extract
    
    Returns:
        Dictionary with analysis results
    """
    print("=" * 80)
    print("TRADEMARK VOCABULARY ANALYSIS")
    print("=" * 80)
    
    # Load dataset
    df = pd.read_csv(csv_path, encoding='MacRoman')
    
    # Extract wordmark columns
    wordmarks = []
    if 'Name RM' in df.columns:
        wordmarks.extend(df['Name RM'].dropna().astype(str).tolist())
    if 'Name TM' in df.columns:
        wordmarks.extend(df['Name TM'].dropna().astype(str).tolist())
    
    print(f"\n📊 Dataset Overview:")
    print(f"   Total trademark pairs: {len(df)}")
    print(f"   Total wordmarks extracted: {len(wordmarks)}")
    
    # Tokenize and count words
    all_words = []
    for mark in wordmarks:
        # Clean and tokenize
        mark = mark.lower()
        # Remove special characters but keep letters
        words = re.findall(r'\b[a-záéíóúâêôãõçüï]+\b', mark)
        all_words.extend(words)
    
    # Count frequencies
    word_freq = Counter(all_words)
    
    print(f"\n📝 Vocabulary Statistics:")
    print(f"   Total words (with repetition): {len(all_words)}")
    print(f"   Unique words: {len(word_freq)}")
    print(f"   Average word length: {sum(len(w) for w in word_freq.keys()) / len(word_freq):.1f} chars")
    
    # Get top N words
    top_words = word_freq.most_common(top_n)
    
    print(f"\n🔝 Top {min(50, len(top_words))} Most Frequent Words:")
    print("-" * 80)
    for i, (word, count) in enumerate(top_words[:50], 1):
        print(f"{i:3d}. {word:20s} → {count:5d} occurrences")
    
    # Categorize words by domain
    print(f"\n🏷️  Categorized Terms:")
    categories = {
        'business': ['empresa', 'comercio', 'loja', 'mercado', 'servicos', 'industria'],
        'quality': ['premium', 'super', 'mega', 'ultra', 'top', 'best', 'melhor'],
        'location': ['brasil', 'rio', 'são', 'paulo', 'nacional', 'internacional'],
        'food': ['cafe', 'food', 'pizza', 'restaurante', 'padaria', 'casa'],
        'fashion': ['moda', 'fashion', 'style', 'wear', 'roupa'],
        'tech': ['tech', 'digital', 'smart', 'web', 'online', 'net'],
    }
    
    for category, keywords in categories.items():
        category_words = [w for w, _ in top_words if any(kw in w for kw in keywords)]
        if category_words:
            print(f"   {category.capitalize():15s}: {', '.join(category_words[:10])}")
    
    # Return analysis results
    return {
        'top_words': top_words,
        'word_freq': word_freq,
        'total_unique': len(word_freq),
        'categorized': categories
    }

# Run the analysis
vocab_analysis = analyze_trademark_vocabulary('trademark_file.csv', top_n=200)

TRADEMARK VOCABULARY ANALYSIS

📊 Dataset Overview:
   Total trademark pairs: 18737
   Total wordmarks extracted: 36718

📝 Vocabulary Statistics:
   Total words (with repetition): 89049
   Unique words: 15302
   Average word length: 6.5 chars

🔝 Top 50 Most Frequent Words:
--------------------------------------------------------------------------------
  1. o                    →  1827 occurrences
  2. e                    →  1457 occurrences
  3. de                   →  1262 occurrences
  4. v                    →   858 occurrences
  5. s                    →   660 occurrences
  6. do                   →   584 occurrences
  7. vôes                 →   468 occurrences
  8. a                    →   459 occurrences
  9. da                   →   382 occurrences
 10. rio                  →   304 occurrences
 11. brasil               →   300 occurrences
 12. casa                 →   276 occurrences
 13. engenharia           →   272 occurrences
 14. avlon                →   266 occurrences
 1

In [14]:
# Latest
##NEWWWWWWWWWWWWWWWWW
import pandas as pd
import numpy as np
from typing import Dict, List, Optional, Tuple
import re
import logging
from dataclasses import dataclass
from deep_translator import GoogleTranslator
import chardet  # For encoding detection
import os
import re

def get_incremented_filename(base_name: str) -> str:
    """
    Returns an incremented filename if the file already exists.
    Example:
      - 'data.csv' → 'data_1.csv' → 'data_2.csv' ...
    """
    if not os.path.exists(base_name):
        return base_name

    # Split name and extension
    name, ext = os.path.splitext(base_name)
    
    # Check for existing numbered versions
    counter = 1
    while True:
        new_name = f"{name}_{counter}{ext}"
        if not os.path.exists(new_name):
            return new_name
        counter += 1

# Add this near the top of your file (after imports)
TRADEMARK_LEXICON = {
    # ========== BUSINESS & COMMERCE ==========
    "company": {"ha": "kamfani", "yo": "ilé-iṣẹ́"},
    "business": {"ha": "kasuwanci", "yo": "iṣẹ́ owó"},
    "commerce": {"ha": "kasuwanci", "yo": "ìṣòwò"},
    "market": {"ha": "kasuwa", "yo": "ọjà"},
    "store": {"ha": "shago", "yo": "ṣọ́ọ̀bù"},
    "shop": {"ha": "shago", "yo": "ṣọ́ọ̀bù"},
    "trade": {"ha": "ciniki", "yo": "ìṣòwò"},
    "service": {"ha": "sabis", "yo": "ìṣẹ́"},
    "services": {"ha": "ayyuka", "yo": "àwọn iṣẹ́"},
    "brand": {"ha": "alama", "yo": "àmì"},
    "logo": {"ha": "tambari", "yo": "àmì"},
    
    # ========== QUALITY DESCRIPTORS ==========
    "premium": {"ha": "mai kyau", "yo": "iyebíye"},
    "super": {"ha": "babba", "yo": "púpọ̀"},
    "mega": {"ha": "babba sosai", "yo": "ńlá púpọ̀"},
    "ultra": {"ha": "mafi", "yo": "jùlọ"},
    "best": {"ha": "mafi kyau", "yo": "dára jùlọ"},
    "top": {"ha": "sama", "yo": "òkè"},
    "elite": {"ha": "zaɓaɓɓu", "yo": "àṣàyàn"},
    "royal": {"ha": "sarauta", "yo": "ọba"},
    "gold": {"ha": "zinariya", "yo": "wúrà"},
    "silver": {"ha": "azurfa", "yo": "fàdákà"},
    "diamond": {"ha": "lu'ulu'u", "yo": "òkúta iyebíye"},
    "original": {"ha": "asali", "yo": "àtọwọ́dọ́wọ́"},
    "classic": {"ha": "na gargajiya", "yo": "ìbílẹ̀"},
    "modern": {"ha": "na zamani", "yo": "ọ̀dẹ"},
    "new": {"ha": "sabon", "yo": "títun"},
    "fresh": {"ha": "sabo", "yo": "tuntun"},
    "natural": {"ha": "na halitta", "yo": "àdáyébá"},
    "organic": {"ha": "kwayoyi na halitta", "yo": "ọganiki"},
    
    # ========== FOOD & BEVERAGE ==========
    "coffee": {"ha": "kofi", "yo": "kọ́fí"},
    "tea": {"ha": "shayi", "yo": "tii"},
    "food": {"ha": "abinci", "yo": "oúnjẹ"},
    "restaurant": {"ha": "gidan cin abinci", "yo": "ilé oúnjẹ"},
    "cafe": {"ha": "kantin kofi", "yo": "ilé kọ́fí"},
    "bakery": {"ha": "gidan burodi", "yo": "ilé búrẹ́dì"},
    "pizza": {"ha": "pizza", "yo": "pítísà"},
    "burger": {"ha": "burger", "yo": "bọ́gà"},
    "bread": {"ha": "burodi", "yo": "búrẹ́dì"},
    "meat": {"ha": "nama", "yo": "ẹran"},
    "milk": {"ha": "madara", "yo": "wàrà"},
    "juice": {"ha": "ruwan 'ya'yan itace", "yo": "omi èso"},
    "water": {"ha": "ruwa", "yo": "omi"},
    "drink": {"ha": "abin sha", "yo": "ohun mímu"},
    "sweet": {"ha": "mai dadi", "yo": "dídùn"},
    "delicious": {"ha": "mai dadi", "yo": "dídùn"},
    
    # ========== FASHION & CLOTHING ==========
    "fashion": {"ha": "salon", "yo": "àṣà aṣọ"},
    "style": {"ha": "salo", "yo": "àṣà"},
    "wear": {"ha": "sawa", "yo": "wọ̀"},
    "clothing": {"ha": "tufafi", "yo": "aṣọ"},
    "dress": {"ha": "riga", "yo": "aṣọ"},
    "shirt": {"ha": "riga", "yo": "ṣẹ́tù"},
    "shoes": {"ha": "takalma", "yo": "bàtà"},
    "hat": {"ha": "hula", "yo": "fìlà"},
    "bag": {"ha": "jaka", "yo": "àpò"},
    "accessories": {"ha": "kayan ado", "yo": "ohun ọ̀ṣọ́"},
    
    # ========== TECHNOLOGY ==========
    "tech": {"ha": "fasaha", "yo": "ìmọ̀-ẹrọ"},
    "technology": {"ha": "fasahar zamani", "yo": "ìmọ̀-ẹrọ"},
    "digital": {"ha": "na dijital", "yo": "oníjìtù"},
    "smart": {"ha": "mai hankali", "yo": "ológbọ́n"},
    "online": {"ha": "kan layi", "yo": "lórí ayélujára"},
    "internet": {"ha": "yanar gizo", "yo": "íńtánẹ́ẹ̀tì"},
    "app": {"ha": "app", "yo": "ìṣàmúlò"},
    "software": {"ha": "software", "yo": "sọ́fìtíwíà"},
    "web": {"ha": "yanar gizo", "yo": "wẹ́ẹ̀bù"},
    "mobile": {"ha": "wayar hannu", "yo": "fóònù alágbèéká"},
    "phone": {"ha": "waya", "yo": "fóònù"},
    "computer": {"ha": "kwamfuta", "yo": "kọ̀mpútà"},
    
    # ========== LOCATION & GEOGRAPHY ==========
    "brazil": {"ha": "Brazil", "yo": "Brazil"},
    "brazilian": {"ha": "na Brazil", "yo": "ará Brazil"},
    "city": {"ha": "birni", "yo": "ìlú"},
    "house": {"ha": "gida", "yo": "ilé"},
    "home": {"ha": "gida", "yo": "ilé"},
    "place": {"ha": "wuri", "yo": "ibì"},
    "center": {"ha": "cibiyar", "yo": "àárín"},
    "plaza": {"ha": "filin", "yo": "gbàgede"},
    
    # ========== SIZE & QUANTITY ==========
    "big": {"ha": "babba", "yo": "ńlá"},
    "small": {"ha": "ƙarami", "yo": "kékeré"},
    "large": {"ha": "babba", "yo": "ńlá"},
    "mini": {"ha": "ƙanƙanta", "yo": "kéékèèké"},
    "maxi": {"ha": "babba sosai", "yo": "tóbi"},
    "extra": {"ha": "ƙari", "yo": "àfikún"},
    "plus": {"ha": "da ƙari", "yo": "àfikún"},
    
    # ========== TIME & STATUS ==========
    "new": {"ha": "sabon", "yo": "títun"},
    "old": {"ha": "tsoho", "yo": "àtijọ́"},
    "now": {"ha": "yanzu", "yo": "báyìí"},
    "today": {"ha": "yau", "yo": "òní"},
    "fast": {"ha": "sauri", "yo": "yára"},
    "quick": {"ha": "sauri", "yo": "yára"},
    "express": {"ha": "gaggawa", "yo": "kíákíá"},
    
    # ========== COLORS ==========
    "red": {"ha": "ja", "yo": "pupa"},
    "blue": {"ha": "shuɗi", "yo": "búlúù"},
    "green": {"ha": "kore", "yo": "aláwọ̀ ewé"},
    "yellow": {"ha": "rawaya", "yo": "oníbùdó"},
    "white": {"ha": "fari", "yo": "funfun"},
    "black": {"ha": "baƙi", "yo": "dúdú"},
    
    # ========== COMMON ACTIONS ==========
    "buy": {"ha": "saya", "yo": "rà"},
    "sell": {"ha": "sayar", "yo": "tà"},
    "make": {"ha": "yi", "yo": "ṣe"},
    "create": {"ha": "ƙirƙira", "yo": "dá"},
    "build": {"ha": "gina", "yo": "kọ́"},
    "design": {"ha": "ƙira", "yo": "yàwòrán"},
    "repair": {"ha": "gyara", "yo": "tún"},
    "fix": {"ha": "gyara", "yo": "tún"},
    
    # ========== COMMON NOUNS ==========
    "king": {"ha": "sarki", "yo": "ọba"},
    "queen": {"ha": "sarauniya", "yo": "ayaba"},
    "people": {"ha": "mutane", "yo": "àwọn ènìyàn"},
    "woman": {"ha": "mace", "yo": "obìnrin"},
    "man": {"ha": "mutum", "yo": "ọkùnrin"},
    "child": {"ha": "yaro", "yo": "ọmọdé"},
    "family": {"ha": "iyali", "yo": "ẹbí"},
    "friend": {"ha": "aboki", "yo": "ọ̀rẹ́"},
    "love": {"ha": "ƙauna", "yo": "ìfẹ́"},
    "life": {"ha": "rayuwa", "yo": "ìgbésí ayé"},
    "world": {"ha": "duniya", "yo": "ayé"},
    
    # Add more based on your dataset analysis...
}

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


# Add this BEFORE the BrazilianTrademarkPreprocessor class

from transformers import pipeline
import warnings
warnings.filterwarnings('ignore')

class EnhancedTranslator:
    """
    Multi-strategy translator with:
    1. Domain-specific lexicon (fastest)
    2. HuggingFace models (better quality)
    3. Google Translate (fallback)
    """
    
    def __init__(self):
        self.lexicon = TRADEMARK_LEXICON
        self.hf_models = {}
        self.google_translators = {
            'ha': GoogleTranslator(source='en', target='ha'),
            'yo': GoogleTranslator(source='en', target='yo')
        }
        self.translation_cache = {}
        self.stats = {'lexicon': 0, 'huggingface': 0, 'google': 0, 'cache': 0}
        
        # Try to load HuggingFace models (optional)
        self._load_hf_models()
    
    def _load_hf_models(self):
        """
        Load HuggingFace translation models for Yoruba/Hausa
        Falls back gracefully if models not available
        """
        try:
            # Facebook's NLLB model supports 200+ languages including Yoruba
            print("Loading HuggingFace models...")
            self.hf_models['yo'] = pipeline(
                "translation", 
                model="facebook/nllb-200-distilled-600M",
                src_lang="eng_Latn",
                tgt_lang="yor_Latn"
            )
            print("✓ Loaded Yoruba model (NLLB)")
        except Exception as e:
            print(f"⚠ HuggingFace Yoruba model not available: {e}")
            print("  Will use Google Translate fallback")
        
        try:
            # Hausa model
            self.hf_models['ha'] = pipeline(
                "translation",
                model="facebook/nllb-200-distilled-600M",
                src_lang="eng_Latn",
                tgt_lang="hau_Latn"
            )
            print("✓ Loaded Hausa model (NLLB)")
        except Exception as e:
            print(f"⚠ HuggingFace Hausa model not available: {e}")
            print("  Will use Google Translate fallback")
    
    def translate_word(self, word: str, target_lang: str) -> str:
        """
        Translate a single word using multi-strategy approach
        Priority: Lexicon → HuggingFace → Google Translate
        """
        if not word or not word.strip():
            return word
        
        word_lower = word.lower().strip()
        cache_key = f"{target_lang}:{word_lower}"
        
        # 1. Check cache
        if cache_key in self.translation_cache:
            self.stats['cache'] += 1
            return self.translation_cache[cache_key]
        
        # 2. Try lexicon (fastest and most accurate for domain terms)
        if word_lower in self.lexicon:
            translation = self.lexicon[word_lower].get(target_lang, word)
            self.stats['lexicon'] += 1
            self.translation_cache[cache_key] = translation
            return translation
        
        # 3. Try HuggingFace model (better than Google for African languages)
        if target_lang in self.hf_models:
            try:
                result = self.hf_models[target_lang](word)
                translation = result[0]['translation_text']
                self.stats['huggingface'] += 1
                self.translation_cache[cache_key] = translation
                return translation
            except Exception as e:
                pass  # Fall through to Google Translate
        
        # 4. Fallback to Google Translate
        try:
            translation = self.google_translators[target_lang].translate(word)
            self.stats['google'] += 1
            self.translation_cache[cache_key] = translation
            return translation
        except Exception as e:
            # Ultimate fallback: return original
            return word
    
    def translate_text(self, text: str, target_lang: str) -> str:
        """
        Translate full text word-by-word
        """
        if pd.isna(text) or not text:
            return ""
        
        words = re.findall(r"\b\w+\b", str(text))
        translated_words = []
        
        for word in words:
            trans = self.translate_word(word, target_lang)
            translated_words.append(trans)
        
        return " ".join(translated_words)
    
    def get_stats(self) -> Dict:
        """Get translation statistics"""
        total = sum(self.stats.values())
        return {
            **self.stats,
            'total': total,
            'lexicon_pct': (self.stats['lexicon'] / total * 100) if total > 0 else 0,
            'hf_pct': (self.stats['huggingface'] / total * 100) if total > 0 else 0,
            'google_pct': (self.stats['google'] / total * 100) if total > 0 else 0,
            'cache_pct': (self.stats['cache'] / total * 100) if total > 0 else 0
        }

# Initialize enhanced translator
enhanced_translator = EnhancedTranslator()
@dataclass
class PreprocessingConfig:
    """Configuration for preprocessing strategy"""
    fix_encoding: bool = True
    translate_to_english: bool = True
    translate_to_local: bool = True  # HA/YO
    remove_duplicates: bool = True
    handle_missing: str = 'drop'  # 'drop' or 'fill'
     # NEW: Sampling control
    max_rows: Optional[int] = None      # e.g., 1000 → process only 1000 rows
    sampling_strategy: str = 'first'    # 'first', 'random', or 'all'


class BrazilianTrademarkPreprocessor:
    """
    Specialized preprocessor for Brazilian INPI trademark dataset
    Handles: Portuguese → English → Hausa/Yoruba translation pipeline
    """
    
    def __init__(self, config: PreprocessingConfig = None):
        self.config = config or PreprocessingConfig()
        self.original_df = None
        self.processed_df = None
        
        # Initialize translators
        self.pt_to_en = GoogleTranslator(source='pt', target='en')
        self.enhanced_translator = EnhancedTranslator()  # NEW: Use enhanced translator
        
        self.translation_cache = {}
    
    def detect_encoding(self, file_path: str) -> str:
        """
        Detect file encoding to fix those v� characters
        """
        with open(file_path, 'rb') as f:
            result = chardet.detect(f.read(100000))  # Read first 100KB
        
        detected = result['encoding']
        confidence = result['confidence']
        
        logger.info(f"Detected encoding: {detected} (confidence: {confidence:.2%})")
        return detected
    
    def _get_lexicon_translation(self, word: str, target_lang: str) -> Optional[str]:
        """
        Get translation from domain lexicon (case-insensitive)
        target_lang: 'ha' or 'yo'
        """
        word_lower = word.lower().strip()
        if word_lower in TRADEMARK_LEXICON:
            return TRADEMARK_LEXICON[word_lower].get(target_lang)
        return None

    def translate_word_with_fallback(self, word: str, target_lang: str) -> str:
        """
        Translate a single word using lexicon first, then Google Translate
        """
        if not word or not word.strip():
            return word
        
        # 1. Try domain lexicon
        lex_trans = self._get_lexicon_translation(word, target_lang)
        if lex_trans:
            return lex_trans
        
        # 2. Fallback to Google Translate
        translator = self.en_to_ha if target_lang == 'ha' else self.en_to_yo
        cache_key = f"en_{target_lang}"
        return self.translate_with_cache(word, translator, cache_key)

    def translate_text_lexicon_first(self, text_en: str, target_lang: str) -> str:
        """
        Translate using enhanced multi-strategy translator
        """
        return self.enhanced_translator.translate_text(text_en, target_lang)

    def load_and_fix_encoding(self, file_path: str) -> pd.DataFrame:
        """
        Load CSV with proper encoding detection and fixing
        """
        # Try to detect encoding
        try:
            encoding = self.detect_encoding(file_path)
        except Exception as e:
            logger.warning(f"Encoding detection failed: {e}. Trying latin-1...")
            encoding = 'latin-1'
        
        # Try loading with detected encoding
        encodings_to_try = [encoding, 'utf-8', 'latin-1', 'cp1252', 'iso-8859-1']
        
        for enc in encodings_to_try:
            try:
                df = pd.read_csv(file_path, encoding=enc)
                logger.info(f"✓ Successfully loaded with encoding: {enc}")
                self.original_df = df
                return df
            except Exception as e:
                logger.debug(f"Failed with {enc}: {e}")
                continue
        
        raise ValueError("Could not load CSV with any common encoding")
    
    def extract_relevant_columns(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Extract and rename relevant columns from Brazilian dataset
        
        Original columns:
        - Process number RM, Name RM, Status RM, etc. (Registered Mark)
        - Process number TM, Name TM, Status TM, etc. (Target Mark - the contested one)
        """
        
        # Map to standardized column names
        column_mapping = {
            # Pair identification
            'Process number RM': 'mark1_id',
            'Process number TM': 'mark2_id',
            
            # Wordmarks (the actual text)
            'Name RM': 'mark1_wordmark',
            'Name TM': 'mark2_wordmark',
            
            # Status
            'Status RM': 'mark1_status',
            'Status TM': 'mark2_status',
            
            # Classification
            'Nice classification RM': 'mark1_class',
            'Nice classification TM': 'mark2_class',
            
            # Presentation (Nominative/Mixed/Figurative)
            'Presentation RM': 'mark1_presentation',
            'Presentation TM': 'mark2_presentation',
            
            # Date
            'Application date RM': 'mark1_date',
            'Application date TM': 'mark2_date',
            
            # Complementary text (explains why similar)
            'Complementary text': 'similarity_reason'
        }
        
        # Select and rename columns
        available_cols = [col for col in column_mapping.keys() if col in df.columns]
        df_processed = df[available_cols].copy()
        df_processed.rename(columns=column_mapping, inplace=True)
        
        # Add label: these pairs are SIMILAR (that's why they're in the rejection dataset)
        df_processed['label'] = 1  # 1 = similar/confusing
        
        logger.info(f"✓ Extracted {len(df_processed)} trademark pairs")
        logger.info(f"   Columns: {list(df_processed.columns)}")
        
        return df_processed
    
    def clean_text(self, text: str) -> str:
        """
        Clean trademark text
        - Remove extra whitespace
        - Fix common encoding artifacts
        - Normalize
        """
        if pd.isna(text) or text == '':
            return ''
        
        text = str(text)
        
        # Fix common encoding issues
        encoding_fixes = {
            'v�': 'ç',
            'v�o': 'ção',
            'v�': 'í',
            'v=': 'ó',
            '?': '',  # Remove question marks from encoding errors
        }
        
        for bad, good in encoding_fixes.items():
            text = text.replace(bad, good)
        
        # Remove extra whitespace
        text = ' '.join(text.split())
        
        return text.strip()
    
    def translate_with_cache(self, text: str, translator, cache_key: str) -> str:
        """
        Translate text with caching to avoid redundant API calls
        """
        if not text or text == '':
            return ''
        
        # Check cache
        cache_id = f"{cache_key}:{text}"
        if cache_id in self.translation_cache:
            return self.translation_cache[cache_id]
        
        # Translate
        try:
            translation = translator.translate(text)
            self.translation_cache[cache_id] = translation
            return translation
        except Exception as e:
            logger.warning(f"Translation failed for '{text[:50]}...': {e}")
            return text  # Return original on failure
    
    def translate_dataset(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Translate wordmarks: Portuguese → English → [Hausa/Yoruba via lexicon + fallback]
        """
        logger.info("🌍 Starting translation pipeline...")
        
        wordmark_cols = ['mark1_wordmark', 'mark2_wordmark']
        
        for col in wordmark_cols:
            if col not in df.columns:
                continue
            
            logger.info(f"   Translating {col}...")
            
            # Portuguese → English (unchanged)
            if self.config.translate_to_english:
                en_col = col.replace('_wordmark', '_wordmark_en')
                df[en_col] = df[col].apply(
                    lambda x: self.translate_with_cache(
                        self.clean_text(x), 
                        self.pt_to_en, 
                        'pt_en'
                    )
                )
                logger.info(f"      ✓ Portuguese → English: {en_col}")
            
            # English → Hausa & Yoruba (enhanced)
            if self.config.translate_to_local:
                # Hausa
                ha_col = col.replace('_wordmark', '_wordmark_ha')
                df[ha_col] = df[en_col].apply(
                    lambda x: self.translate_text_lexicon_first(str(x), 'ha')
                )
                logger.info(f"      ✓ English → Hausa (lexicon+fallback): {ha_col}")
                
                # Yoruba
                yo_col = col.replace('_wordmark', '_wordmark_yo')
                df[yo_col] = df[en_col].apply(
                    lambda x: self.translate_text_lexicon_first(str(x), 'yo')
                )
                logger.info(f"      ✓ English → Yoruba (lexicon+fallback): {yo_col}")
        
        return df
    
    def handle_missing_values(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Handle missing values based on configuration
        """
        initial_count = len(df)
        
        if self.config.handle_missing == 'drop':
            # Drop rows where critical columns are missing
            critical_cols = ['mark1_wordmark', 'mark2_wordmark']
            df = df.dropna(subset=critical_cols)
            removed = initial_count - len(df)
            if removed > 0:
                logger.info(f"⚠ Removed {removed} rows with missing wordmarks")
        
        return df
    
    def remove_duplicates(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Remove duplicate trademark pairs
        """
        if not self.config.remove_duplicates:
            return df
        
        initial_count = len(df)
        
        # Sort pairs to avoid (A,B) vs (B,A) duplicates
        df['pair_key'] = df.apply(
            lambda row: tuple(sorted([
                str(row['mark1_wordmark']), 
                str(row['mark2_wordmark'])
            ])),
            axis=1
        )
        
        df = df.drop_duplicates(subset='pair_key')
        df = df.drop('pair_key', axis=1)
        
        removed = initial_count - len(df)
        if removed > 0:
            logger.info(f"⚠ Removed {removed} duplicate pairs")
        
        return df
    
    def add_features(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Add basic features for later use
        """
        # Length features
        df['mark1_length'] = df['mark1_wordmark'].str.len()
        df['mark2_length'] = df['mark2_wordmark'].str.len()
        
        # Same class indicator
        df['same_class'] = (df['mark1_class'] == df['mark2_class']).astype(int)
        
        # Length difference
        df['length_diff'] = abs(df['mark1_length'] - df['mark2_length'])
        
        return df
    
    def _sample_dataframe(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Sample the dataframe based on config: first N, random N, or all.
        Applied AFTER loading but BEFORE heavy processing (cleaning, translation).
        """

        if self.config.max_rows is not None and self.config.max_rows <= 0:
            raise ValueError("max_rows must be positive integer or None")
        
        if self.config.sampling_strategy == 'all' or self.config.max_rows is None:
            logger.info("📝 Sampling strategy: using all rows")
            return df

        n = min(self.config.max_rows, len(df))
        
        if self.config.sampling_strategy == 'first':
            sampled_df = df.head(n).copy()
            logger.info(f"📝 Sampling strategy: first {n} rows")
            
        elif self.config.sampling_strategy == 'random':
            sampled_df = df.sample(n=n, random_state=42).copy()  # reproducible
            logger.info(f"📝 Sampling strategy: random {n} rows (seed=42)")
            
        else:
            raise ValueError(f"Unknown sampling strategy: {self.config.sampling_strategy}")
        
        return sampled_df

    def process(self, file_path: str) -> pd.DataFrame:
        """
        Full preprocessing pipeline
        """
        logger.info("=" * 80)
        logger.info("BRAZILIAN TRADEMARK DATASET PREPROCESSING")
        logger.info("=" * 80)
        
        # 1. Load with encoding fix
        logger.info("\n1️⃣ Loading dataset...")
        df = self.load_and_fix_encoding(file_path)
        logger.info(f"   Loaded: {df.shape}")
        
        # 2. Extract relevant columns
        logger.info("\n2️⃣ Extracting relevant columns...")
        df = self.extract_relevant_columns(df)
        
        # >>> NEW: Apply sampling here <<<
        logger.info("\n3️⃣ Applying sampling strategy...")
        df = self._sample_dataframe(df)
        logger.info(f"   After sampling: {df.shape}")

        # 3. Clean text
        logger.info("\n3️⃣ Cleaning text...")
        for col in df.columns:
            if 'wordmark' in col or 'reason' in col:
                df[col] = df[col].apply(self.clean_text)
        
        # 4. Handle missing values
        logger.info("\n4️⃣ Handling missing values...")
        df = self.handle_missing_values(df)
        
        # 5. Remove duplicates
        logger.info("\n5️⃣ Removing duplicates...")
        df = self.remove_duplicates(df)
        
        # 6. Translate
        logger.info("\n6️⃣ Translating dataset...")
        df = self.translate_dataset(df)
        
        # 7. Add features
        logger.info("\n7️⃣ Adding features...")
        df = self.add_features(df)
        
        self.processed_df = df
        
        logger.info("\n" + "=" * 80)
        logger.info(f"✅ PREPROCESSING COMPLETE")
        logger.info(f"   Final dataset: {df.shape}")
        logger.info(f"   Columns: {list(df.columns)}")
        logger.info("=" * 80)
        
        # After processing, show translation stats
        stats = self.enhanced_translator.get_stats()
        logger.info("\n📊 Translation Statistics:")
        logger.info(f"   Lexicon hits: {stats['lexicon']} ({stats['lexicon_pct']:.1f}%)")
        logger.info(f"   HuggingFace: {stats['huggingface']} ({stats['hf_pct']:.1f}%)")
        logger.info(f"   Google Translate: {stats['google']} ({stats['google_pct']:.1f}%)")
        logger.info(f"   Cache hits: {stats['cache']} ({stats['cache_pct']:.1f}%)")
        
        return df
    
    def get_sample_report(self, n: int = 10) -> pd.DataFrame:
        """
        Get sample of processed data for inspection
        """
        if self.processed_df is None:
            raise ValueError("No processed data. Run process() first.")
        
        cols_to_show = [
            'mark1_wordmark', 'mark1_wordmark_en', 'mark1_wordmark_ha', 'mark1_wordmark_yo',
            'mark2_wordmark', 'mark2_wordmark_en', 'mark2_wordmark_ha', 'mark2_wordmark_yo',
            'label', 'same_class'
        ]
        
        available_cols = [col for col in cols_to_show if col in self.processed_df.columns]
        
        return self.processed_df[available_cols].head(n)
    
    def save_processed(self, output_path: str):
        """
        Save processed dataset
        """
        if self.processed_df is None:
            raise ValueError("No processed data to save")
        
        self.processed_df.to_csv(output_path, index=False, encoding='utf-8')
        logger.info(f"✅ Saved processed dataset to: {output_path}")


    # Add these methods to the BrazilianTrademarkPreprocessor class

    def generate_negative_pairs(
        self, 
        df: pd.DataFrame, 
        ratio: float = 1.0,
        strategy: str = 'mixed'  # 'random', 'class_based', 'mixed'
    ) -> pd.DataFrame:
        """
        Generate negative (dissimilar) pairs from positive pairs
        
        Args:
            df: DataFrame with positive pairs (label=1)
            ratio: How many negative pairs to generate (1.0 = same as positive)
            strategy: 'random', 'class_based', or 'mixed'
        
        Returns:
            DataFrame with negative pairs (label=0)
        """
        logger.info(f"\n🔄 Generating negative pairs (strategy: {strategy}, ratio: {ratio})...")
        
        # Get unique wordmarks for each mark position
        mark1_pool = df['mark1_wordmark'].unique()
        mark2_pool = df['mark2_wordmark'].unique()
        
        # Combine all unique wordmarks and filter out empty/invalid ones
        all_wordmarks = list(set(list(mark1_pool) + list(mark2_pool)))
        all_wordmarks = [
            str(w).strip() for w in all_wordmarks 
            if pd.notna(w) and str(w).strip() and len(str(w).strip()) > 0
        ]
        
        if len(all_wordmarks) < 2:
            logger.error(f"❌ Not enough valid wordmarks to generate pairs. Found: {len(all_wordmarks)}")
            return pd.DataFrame()
        
        logger.info(f"   Valid wordmarks available: {len(all_wordmarks)}")
        
        # Get class information if available
        mark1_classes = df.set_index('mark1_wordmark')['mark1_class'].to_dict()
        mark2_classes = df.set_index('mark2_wordmark')['mark2_class'].to_dict()
        all_classes = {**mark1_classes, **mark2_classes}
        
        # Calculate number of negative pairs needed
        n_positive = len(df)
        n_negative = int(n_positive * ratio)
        
        logger.info(f"   Positive pairs: {n_positive}")
        logger.info(f"   Target negative pairs: {n_negative}")
        
        negative_pairs = []
        max_attempts = n_negative * 50  # Increased attempts for better success rate
        attempts = 0
        consecutive_fails = 0
        max_consecutive_fails = 100
        
        # Track existing pairs to avoid duplicates
        existing_pairs = set()
        for _, row in df.iterrows():
            try:
                mark1_str = str(row['mark1_wordmark']).strip()
                mark2_str = str(row['mark2_wordmark']).strip()
                if mark1_str and mark2_str:
                    pair = tuple(sorted([mark1_str, mark2_str]))
                    existing_pairs.add(pair)
            except Exception as e:
                logger.debug(f"Skipping invalid pair: {e}")
                continue
        
        logger.info(f"   Existing pairs to avoid: {len(existing_pairs)}")
        
        while len(negative_pairs) < n_negative and attempts < max_attempts:
            attempts += 1
            
            try:
                # Random selection
                mark1 = np.random.choice(all_wordmarks)
                mark2 = np.random.choice(all_wordmarks)
                
                # Validate marks
                if not mark1 or not mark2 or not isinstance(mark1, str) or not isinstance(mark2, str):
                    consecutive_fails += 1
                    continue
                
                mark1 = mark1.strip()
                mark2 = mark2.strip()
                
                # Skip if empty
                if not mark1 or not mark2:
                    consecutive_fails += 1
                    continue
                
                # Skip if same
                if mark1.lower() == mark2.lower():
                    consecutive_fails += 1
                    continue
                
                # Create sorted pair to check for duplicates
                pair_key = tuple(sorted([mark1, mark2]))
                
                # Skip if already exists (either as positive or already generated negative)
                if pair_key in existing_pairs:
                    consecutive_fails += 1
                    continue
                
                # Apply strategy-specific filtering
                if strategy == 'class_based':
                    # Only accept if different classes
                    class1 = all_classes.get(mark1, 0)
                    class2 = all_classes.get(mark2, 0)
                    if class1 == class2:
                        consecutive_fails += 1
                        continue
                
                elif strategy == 'mixed':
                    # Accept if: different class OR very different length OR different first letter
                    class1 = all_classes.get(mark1, 0)
                    class2 = all_classes.get(mark2, 0)
                    len1 = len(mark1)
                    len2 = len(mark2)
                    
                    # Safely get first character
                    try:
                        first1 = mark1[0].lower() if len(mark1) > 0 else ''
                        first2 = mark2[0].lower() if len(mark2) > 0 else ''
                    except (IndexError, AttributeError):
                        first1 = ''
                        first2 = ''
                    
                    # Must satisfy at least one dissimilarity criterion
                    if not (class1 != class2 or abs(len1 - len2) > 3 or (first1 and first2 and first1 != first2)):
                        consecutive_fails += 1
                        continue
                
                # If we get here, it's a valid negative pair
                existing_pairs.add(pair_key)
                consecutive_fails = 0  # Reset counter
                
                negative_pairs.append({
                    'mark1_wordmark': mark1,
                    'mark2_wordmark': mark2,
                    'mark1_class': all_classes.get(mark1, 0),
                    'mark2_class': all_classes.get(mark2, 0),
                    'label': 0,  # Dissimilar
                    'pair_type': 'negative'
                })
                
                # Progress indicator
                if len(negative_pairs) % 10 == 0:
                    logger.debug(f"   Generated {len(negative_pairs)}/{n_negative} pairs (attempt {attempts})")
            
            except Exception as e:
                logger.debug(f"Error generating pair at attempt {attempts}: {e}")
                consecutive_fails += 1
                continue
            
            # Safety check: if too many consecutive failures, stop
            if consecutive_fails >= max_consecutive_fails:
                logger.warning(f"⚠ Stopping after {consecutive_fails} consecutive failures")
                break
        
        if len(negative_pairs) == 0:
            logger.error(f"❌ Failed to generate any negative pairs after {attempts} attempts")
            logger.error(f"   Available wordmarks: {len(all_wordmarks)}")
            logger.error(f"   Sample wordmarks: {all_wordmarks[:5] if len(all_wordmarks) >= 5 else all_wordmarks}")
            return pd.DataFrame()
        
        if len(negative_pairs) < n_negative:
            logger.warning(f"⚠ Only generated {len(negative_pairs)}/{n_negative} negative pairs after {attempts} attempts")
        else:
            logger.info(f"✓ Generated {len(negative_pairs)} negative pairs in {attempts} attempts")
        
        negative_df = pd.DataFrame(negative_pairs)
        return negative_df


    def create_balanced_dataset(
        self,
        positive_df: pd.DataFrame,
        negative_ratio: float = 1.0,
        strategy: str = 'mixed'
    ) -> pd.DataFrame:
        """
        Create a balanced dataset with both positive and negative pairs
        
        Args:
            positive_df: DataFrame with positive pairs
            negative_ratio: Ratio of negative to positive pairs
            strategy: Strategy for generating negatives
        
        Returns:
            Combined balanced dataset (or just positive_df if negative generation fails)
        """
        logger.info("\n📊 Creating balanced dataset...")
        
        # Validate input
        if positive_df is None or len(positive_df) == 0:
            logger.error("❌ No positive pairs provided")
            return pd.DataFrame()
        
        # Add pair_type to positive pairs
        positive_df = positive_df.copy()
        positive_df['pair_type'] = 'positive'
        
        # Generate negative pairs
        try:
            negative_df = self.generate_negative_pairs(
                positive_df, 
                ratio=negative_ratio, 
                strategy=strategy
            )
        except Exception as e:
            logger.error(f"❌ Failed to generate negative pairs: {e}")
            logger.warning("⚠ Returning only positive pairs")
            return positive_df
        
        # Check if negative generation succeeded
        if negative_df is None or len(negative_df) == 0:
            logger.warning("⚠ No negative pairs generated. Returning only positive pairs")
            return positive_df
        
        # Align columns
        # For negative pairs, we don't have English/HA/YO translations yet
        # So we'll need to translate them too
        if 'mark1_wordmark_en' in positive_df.columns:
            try:
                logger.info("   Translating negative pairs...")
                negative_df = self._translate_negative_pairs(negative_df)
            except Exception as e:
                logger.error(f"❌ Translation failed for negative pairs: {e}")
                logger.warning("⚠ Returning only positive pairs")
                return positive_df
        
        # Add missing columns from positive pairs
        for col in positive_df.columns:
            if col not in negative_df.columns:
                try:
                    if 'mark1' in col and col != 'mark1_wordmark' and col != 'mark1_class':
                        negative_df[col] = negative_df['mark1_wordmark'].map(
                            positive_df.set_index('mark1_wordmark')[col].to_dict()
                        )
                    elif 'mark2' in col and col != 'mark2_wordmark' and col != 'mark2_class':
                        negative_df[col] = negative_df['mark2_wordmark'].map(
                            positive_df.set_index('mark2_wordmark')[col].to_dict()
                        )
                    else:
                        negative_df[col] = None
                except Exception as e:
                    logger.debug(f"Could not map column {col}: {e}")
                    negative_df[col] = None
        
        # Combine
        try:
            balanced_df = pd.concat([positive_df, negative_df], ignore_index=True)
            
            # Shuffle
            balanced_df = balanced_df.sample(frac=1, random_state=42).reset_index(drop=True)
            
            logger.info(f"✓ Balanced dataset created:")
            logger.info(f"   Positive pairs: {len(positive_df)} ({len(positive_df)/len(balanced_df)*100:.1f}%)")
            logger.info(f"   Negative pairs: {len(negative_df)} ({len(negative_df)/len(balanced_df)*100:.1f}%)")
            logger.info(f"   Total: {len(balanced_df)}")
            
            return balanced_df
        
        except Exception as e:
            logger.error(f"❌ Failed to combine datasets: {e}")
            logger.warning("⚠ Returning only positive pairs")
            return positive_df


    def _translate_negative_pairs(self, negative_df: pd.DataFrame) -> pd.DataFrame:
        """
        Translate negative pairs using the same pipeline as positive pairs
        """
        if negative_df is None or len(negative_df) == 0:
            return negative_df
        
        negative_df = negative_df.copy()
        
        try:
            # Build translation dictionaries from cache
            for col in ['mark1_wordmark', 'mark2_wordmark']:
                base_col = col
                
                try:
                    # English
                    en_col = col.replace('_wordmark', '_wordmark_en')
                    negative_df[en_col] = negative_df[base_col].apply(
                        lambda x: self.translate_with_cache(
                            self.clean_text(str(x)), 
                            self.pt_to_en, 
                            'pt_en'
                        ) if pd.notna(x) and str(x).strip() else ''
                    )
                except Exception as e:
                    logger.warning(f"English translation failed for {col}: {e}")
                    negative_df[en_col] = negative_df[base_col]
                
                try:
                    # Hausa
                    ha_col = col.replace('_wordmark', '_wordmark_ha')
                    negative_df[ha_col] = negative_df[en_col].apply(
                        lambda x: self.translate_text_lexicon_first(str(x), 'ha') 
                        if pd.notna(x) and str(x).strip() else ''
                    )
                except Exception as e:
                    logger.warning(f"Hausa translation failed for {col}: {e}")
                    negative_df[ha_col] = negative_df[en_col]
                
                try:
                    # Yoruba
                    yo_col = col.replace('_wordmark', '_wordmark_yo')
                    negative_df[yo_col] = negative_df[en_col].apply(
                        lambda x: self.translate_text_lexicon_first(str(x), 'yo')
                        if pd.notna(x) and str(x).strip() else ''
                    )
                except Exception as e:
                    logger.warning(f"Yoruba translation failed for {col}: {e}")
                    negative_df[yo_col] = negative_df[en_col]
        
        except Exception as e:
            logger.error(f"❌ Critical error in translation: {e}")
            raise
        
        return negative_df


    def analyze_dataset_balance(self, df: pd.DataFrame) -> Dict:
        """
        Analyze the balance and characteristics of the dataset
        """
        print("\n" + "=" * 80)
        print("DATASET BALANCE ANALYSIS")
        print("=" * 80)
        
        # Label distribution
        label_dist = df['label'].value_counts()
        print(f"\n📊 Label Distribution:")
        for label, count in label_dist.items():
            label_name = "Similar (Positive)" if label == 1 else "Dissimilar (Negative)"
            print(f"   {label_name}: {count} ({count/len(df)*100:.1f}%)")
        
        # Pair type distribution
        if 'pair_type' in df.columns:
            pair_dist = df['pair_type'].value_counts()
            print(f"\n🔄 Pair Type Distribution:")
            for ptype, count in pair_dist.items():
                print(f"   {ptype.capitalize()}: {count} ({count/len(df)*100:.1f}%)")
        
        # Class overlap
        same_class = df['same_class'].value_counts()
        print(f"\n🏷️  Same Class Distribution:")
        for same, count in same_class.items():
            class_label = "Same Class" if same == 1 else "Different Class"
            print(f"   {class_label}: {count} ({count/len(df)*100:.1f}%)")
        
        # Length statistics by label
        print(f"\n📏 Length Statistics by Label:")
        for label in [0, 1]:
            subset = df[df['label'] == label]
            label_name = "Positive" if label == 1 else "Negative"
            avg_len1 = subset['mark1_length'].mean()
            avg_len2 = subset['mark2_length'].mean()
            avg_diff = subset['length_diff'].mean()
            print(f"   {label_name} pairs:")
            print(f"      Avg mark1 length: {avg_len1:.1f}")
            print(f"      Avg mark2 length: {avg_len2:.1f}")
            print(f"      Avg length diff: {avg_diff:.1f}")
        
        return {
            'total_pairs': len(df),
            'positive_pairs': label_dist.get(1, 0),
            'negative_pairs': label_dist.get(0, 0),
            'balance_ratio': label_dist.get(0, 0) / label_dist.get(1, 1) if label_dist.get(1, 0) > 0 else 0
        }


# ===========================
# ENHANCED USAGE WITH NEGATIVE PAIRS
# ===========================

# Configure preprocessing
config = PreprocessingConfig(
    max_rows=5,  # Process first 100 rows (or use None for all)
    sampling_strategy='random',  # 'first', 'random', or 'all'
    fix_encoding=True,
    translate_to_english=True,
    translate_to_local=True,  # Translate to HA/YO
    remove_duplicates=True,
    handle_missing='drop'
)

# Initialize preprocessor
preprocessor = BrazilianTrademarkPreprocessor(config)

# Process the dataset
try:
    # Step 1: Process positive pairs (similar trademarks)
    print("=" * 80)
    print("STEP 1: PROCESSING POSITIVE PAIRS (Similar Trademarks)")
    print("=" * 80)
    positive_df = preprocessor.process('trademark_file.csv')
    
    if positive_df is None or len(positive_df) == 0:
        raise ValueError("No positive pairs generated from preprocessing")
    
    print(f"\n✓ Positive pairs ready: {len(positive_df)}")
    
    # Step 2: Generate negative pairs (dissimilar trademarks)
    print("\n" + "=" * 80)
    print("STEP 2: GENERATING NEGATIVE PAIRS (Dissimilar Trademarks)")
    print("=" * 80)
    
    # Create balanced dataset with negative pairs
    balanced_df = preprocessor.create_balanced_dataset(
        positive_df,
        negative_ratio=1.0,  # 1:1 ratio (equal positive and negative)
        strategy='mixed'  # 'random', 'class_based', or 'mixed'
    )
    
    if balanced_df is None or len(balanced_df) == 0:
        raise ValueError("No balanced dataset generated")
    
    # Check if we actually have both positive and negative
    has_negatives = 0 in balanced_df['label'].values
    
    if not has_negatives:
        logger.warning("⚠ Dataset contains only positive pairs (no negatives generated)")
        logger.info("   This can still be used, but won't be balanced")
    
    # Step 3: Analyze balance
    print("\n" + "=" * 80)
    print("STEP 3: ANALYZING DATASET BALANCE")
    print("=" * 80)
    balance_stats = preprocessor.analyze_dataset_balance(balanced_df)
    
    # Step 4: Show sample from both classes
    print("\n" + "=" * 80)
    print("SAMPLE DATA")
    print("=" * 80)
    
    if 1 in balanced_df['label'].values:
        print("\n🟢 POSITIVE PAIRS (Similar - Label=1):")
        positive_sample = balanced_df[balanced_df['label'] == 1].head(3)
        print(positive_sample[['mark1_wordmark', 'mark2_wordmark', 'label', 'same_class']].to_string(index=False))
    
    if 0 in balanced_df['label'].values:
        print("\n🔴 NEGATIVE PAIRS (Dissimilar - Label=0):")
        negative_sample = balanced_df[balanced_df['label'] == 0].head(3)
        print(negative_sample[['mark1_wordmark', 'mark2_wordmark', 'label', 'same_class']].to_string(index=False))
    
    # Step 5: Save balanced dataset
    base_output_file = 'trademark_data_balanced_multilingual.csv'
    output_file = get_incremented_filename(base_output_file)

    balanced_df.to_csv(output_file, index=False, encoding='utf-8')

    print(f"\n✅ Dataset saved to: {output_file}")
    print(f"   Total pairs: {len(balanced_df)}")
    print(f"   Columns: {len(balanced_df.columns)}")
    print(f"   File size: {balanced_df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
    
    # Step 6: Save comprehensive summary statistics
    summary = {
        'dataset_info': {
            'total_pairs': int(len(balanced_df)),
            'positive_pairs': int(balance_stats['positive_pairs']),
            'negative_pairs': int(balance_stats['negative_pairs']),
            'balance_ratio': float(balance_stats['balance_ratio']),
            'file_size_mb': float(balanced_df.memory_usage(deep=True).sum() / 1024**2),
            'has_negative_pairs': has_negatives
        },
        'preprocessing': {
            'languages': ['Portuguese', 'English', 'Hausa', 'Yoruba'],
            'max_rows_processed': config.max_rows if config.max_rows else 'all',
            'sampling_strategy': config.sampling_strategy,
            'negative_generation_strategy': 'mixed',
            'negative_ratio': 1.0
        },
        'columns': {
            'total_columns': len(balanced_df.columns),
            'column_list': list(balanced_df.columns)
        },
        'class_distribution': {
            'same_class_pairs': int(balanced_df['same_class'].sum()),
            'different_class_pairs': int((balanced_df['same_class'] == 0).sum()),
            'same_class_percentage': float(balanced_df['same_class'].mean() * 100)
        },
        'length_statistics': {
            'avg_mark1_length': float(balanced_df['mark1_length'].mean()),
            'avg_mark2_length': float(balanced_df['mark2_length'].mean()),
            'avg_length_diff': float(balanced_df['length_diff'].mean())
        }
    }

    import json
    with open('dataset_summary.json', 'w', encoding='utf-8') as f:
        json.dump(summary, f, indent=2, ensure_ascii=False)

    print("\n✅ Dataset ready for Step 1 (CNN + SVM training)")
    print(f"   Use file: {output_file}")
    print(f"   Summary saved to: dataset_summary.json")

    # Also print the summary
    print("\n📄 Summary:")
    print(json.dumps(summary, indent=2))
    
except Exception as e:
    print(f"\n❌ Error: {e}")
    import traceback
    traceback.print_exc()
    
    # Try to save whatever we have
    if 'positive_df' in locals() and positive_df is not None and len(positive_df) > 0:
        emergency_file = 'trademark_data_positive_only.csv'
        positive_df.to_csv(emergency_file, index=False, encoding='utf-8')
        print(f"\n⚠ Saved positive pairs only to: {emergency_file}")

Loading HuggingFace models...


Device set to use cpu


✓ Loaded Yoruba model (NLLB)


Device set to use cpu


✓ Loaded Hausa model (NLLB)
Loading HuggingFace models...


...

✓ Loaded Hausa model (NLLB)


INFO:__main__:================================================================================
INFO:__main__:BRAZILIAN TRADEMARK DATASET PREPROCESSING
INFO:__main__:================================================================================
INFO:__main__:
1️⃣ Loading dataset...
INFO:__main__:Detected encoding: MacRoman (confidence: 72.30%)


STEP 1: PROCESSING POSITIVE PAIRS (Similar Trademarks)


INFO:__main__:✓ Successfully loaded with encoding: MacRoman
INFO:__main__:   Loaded: (18737, 18)
INFO:__main__:
2️⃣ Extracting relevant columns...
INFO:__main__:✓ Extracted 18737 trademark pairs
INFO:__main__:   Columns: ['mark1_id', 'mark2_id', 'mark1_wordmark', 'mark2_wordmark', 'mark1_status', 'mark2_status', 'mark1_class', 'mark2_class', 'mark1_presentation', 'mark2_presentation', 'mark1_date', 'mark2_date', 'similarity_reason', 'label']
INFO:__main__:
3️⃣ Applying sampling strategy...
INFO:__main__:📝 Sampling strategy: random 5 rows (seed=42)
INFO:__main__:   After sampling: (5, 14)
INFO:__main__:
3️⃣ Cleaning text...
INFO:__main__:
4️⃣ Handling missing values...
INFO:__main__:
5️⃣ Removing duplicates...
INFO:__main__:
6️⃣ Translating dataset...
INFO:__main__:🌍 Starting translation pipeline...
INFO:__main__:   Translating mark1_wordmark...
INFO:__main__:      ✓ Portuguese → English: mark1_wordmark_en
INFO:__main__:      ✓ English → Hausa (lexicon+fallback): mark1_wordmark_ha
INFO:


✓ Positive pairs ready: 5

STEP 2: GENERATING NEGATIVE PAIRS (Dissimilar Trademarks)

STEP 3: ANALYZING DATASET BALANCE

DATASET BALANCE ANALYSIS

📊 Label Distribution:
   Dissimilar (Negative): 5 (50.0%)
   Similar (Positive): 5 (50.0%)

🔄 Pair Type Distribution:
   Negative: 5 (50.0%)
   Positive: 5 (50.0%)

🏷️  Same Class Distribution:
   Different Class: 4 (40.0%)
   Same Class: 1 (10.0%)

📏 Length Statistics by Label:
   Negative pairs:
      Avg mark1 length: 8.0
      Avg mark2 length: 11.2
      Avg length diff: nan
   Positive pairs:
      Avg mark1 length: 11.2
      Avg mark2 length: 9.0
      Avg length diff: 4.2

SAMPLE DATA

🟢 POSITIVE PAIRS (Similar - Label=1):
mark1_wordmark   mark2_wordmark  label same_class
  LIDER DESIGN LvÁDER ESTOFADOS      1          0
       KOLENZA          COSENZA      1          0
      SECPOWER        Power Sek      1          1

🔴 NEGATIVE PAIRS (Dissimilar - Label=0):
mark1_wordmark mark2_wordmark  label same_class
     Power Sek  Digital 

In [7]:
pip install pandas numpy scikit-learn jellyfish sentence-transformers torch

   ---------------------------------------- 0.0/8.0 MB ? eta -:--:--
   ------ --------------------------------- 1.3/8.0 MB 7.1 MB/s eta 0:00:01
   ------------------ --------------------- 3.7/8.0 MB 11.3 MB/s eta 0:00:01
   ---------------------------- ----------- 5.8/8.0 MB 10.3 MB/s eta 0:00:01
   -------------------------------------- - 7.6/8.0 MB 9.9 MB/s eta 0:00:01
   ---------------------------------------- 8.0/8.0 MB 9.8 MB/s  0:00:00
   ---------------------------------------- 0.0/36.3 MB ? eta -:--:--
   -- ------------------------------------- 2.4/36.3 MB 11.8 MB/s eta 0:00:03
   ----- ---------------------------------- 4.7/36.3 MB 11.7 MB/s eta 0:00:03
   ------- -------------------------------- 6.8/36.3 MB 11.0 MB/s eta 0:00:03
   --------- ------------------------------ 8.4/36.3 MB 10.2 MB/s eta 0:00:03
   ----------- ---------------------------- 10.5/36.3 MB 10.0 MB/s eta 0:00:03
   ------------- -------------------------- 12.6/36.3 MB 9.9 MB/s eta 0:00:03
   ----------


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
!pip install tf-keras

   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ------------------------ --------------- 1.0/1.7 MB 6.7 MB/s eta 0:00:01
   ---------------------------------------- 1.7/1.7 MB 6.5 MB/s  0:00:00



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
# ===========================
# STEP 1.5: FEATURE ENGINEERING
# ===========================

import pandas as pd
import jellyfish  # For string distance + phonetic algorithms
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

class AdvancedFeatureExtractor:
    """
    Extract sight/sound/meaning features aligned with legal standards
    """
    
    def __init__(self):
        # Load once, reuse
        self.semantic_model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
    
    def _safe_encode(self, texts: list) -> np.ndarray:
        """Handle NaN/empty strings during encoding"""
        clean_texts = []
        for t in texts:
            if pd.isna(t) or t == '' or t is None:
                clean_texts.append("")  # Model handles empty string gracefully
            else:
                clean_texts.append(str(t))
        return self.semantic_model.encode(clean_texts, show_progress_bar=False)
    
    def extract_all(self, df: pd.DataFrame) -> pd.DataFrame:
        """Extract all features"""
        df = df.copy()  # Avoid modifying original
        
        print("🔍 Extracting visual features...")
        # 1. SIGHT (Visual) - Character-level
        df['visual_levenshtein'] = df.apply(
            lambda row: jellyfish.levenshtein_distance(
                str(row['mark1_wordmark']), str(row['mark2_wordmark'])
            ), axis=1
        )
        df['visual_jaro_winkler'] = df.apply(
            lambda row: jellyfish.jaro_winkler_similarity(
                str(row['mark1_wordmark']), str(row['mark2_wordmark'])
            ), axis=1
        )
        
        print("👂 Extracting phonetic features...")
        # 2. SOUND (Phonetic) – using jellyfish
        df['soundex_match'] = df.apply(
            lambda row: int(
                jellyfish.soundex(str(row['mark1_wordmark'])) == 
                jellyfish.soundex(str(row['mark2_wordmark']))
            ), axis=1
        )
        df['metaphone_match'] = df.apply(
            lambda row: int(
                jellyfish.metaphone(str(row['mark1_wordmark'])) == 
                jellyfish.metaphone(str(row['mark2_wordmark']))
            ), axis=1
        )
        
        print("🧠 Extracting semantic features (English)...")
        # 3. MEANING (Semantic) – English
        mark1_emb_en = self._safe_encode(df['mark1_wordmark_en'].tolist())
        mark2_emb_en = self._safe_encode(df['mark2_wordmark_en'].tolist())
        df['semantic_similarity_en'] = [
            float(cosine_similarity([e1], [e2])[0][0]) 
            for e1, e2 in zip(mark1_emb_en, mark2_emb_en)
        ]
        
        print("🧠 Extracting semantic features (Hausa)...")
        mark1_emb_ha = self._safe_encode(df['mark1_wordmark_ha'].tolist())
        mark2_emb_ha = self._safe_encode(df['mark2_wordmark_ha'].tolist())
        df['semantic_similarity_ha'] = [
            float(cosine_similarity([e1], [e2])[0][0]) 
            for e1, e2 in zip(mark1_emb_ha, mark2_emb_ha)
        ]
        
        print("🧠 Extracting semantic features (Yoruba)...")
        mark1_emb_yo = self._safe_encode(df['mark1_wordmark_yo'].tolist())
        mark2_emb_yo = self._safe_encode(df['mark2_wordmark_yo'].tolist())
        df['semantic_similarity_yo'] = [
            float(cosine_similarity([e1], [e2])[0][0]) 
            for e1, e2 in zip(mark1_emb_yo, mark2_emb_yo)
        ]
        
        return df

# Usage
feature_extractor = AdvancedFeatureExtractor()
balanced_df = feature_extractor.extract_all(balanced_df)

print("\n✅ Feature extraction complete!")
new_features = [col for col in balanced_df.columns if any(kw in col for kw in ['visual', 'soundex', 'metaphone', 'semantic'])]
print(f"   New feature columns ({len(new_features)}): {new_features}")

# Use a descriptive name
output_file = get_incremented_filename('trademark_data_balanced_multilingual_with_features.csv')
balanced_df.to_csv(output_file, index=False, encoding='utf-8')

print(f"\n💾 Enhanced dataset saved to: {output_file}")
print(f"   Total rows: {len(balanced_df)}")
print(f"   Total columns: {len(balanced_df.columns)}")
print(f"   New feature columns: {[col for col in balanced_df.columns if 'visual' in col or 'soundex' in col or 'semantic' in col]}")

INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cpu
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: paraphrase-multilingual-MiniLM-L12-v2


🔍 Extracting visual features...
👂 Extracting phonetic features...
🧠 Extracting semantic features (English)...
🧠 Extracting semantic features (Hausa)...
🧠 Extracting semantic features (Yoruba)...

✅ Feature extraction complete!
   New feature columns (7): ['visual_levenshtein', 'visual_jaro_winkler', 'soundex_match', 'metaphone_match', 'semantic_similarity_en', 'semantic_similarity_ha', 'semantic_similarity_yo']

💾 Enhanced dataset saved to: trademark_data_balanced_multilingual_with_features.csv
   Total rows: 8
   Total columns: 32
   New feature columns: ['visual_levenshtein', 'visual_jaro_winkler', 'soundex_match', 'semantic_similarity_en', 'semantic_similarity_ha', 'semantic_similarity_yo']
